<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/A_05_24_DNA_Library_(EXP_6_BW_chart).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install python-gedcom
!pip install openpyxl

In [5]:
#A_v_2024-05-22_produces CORRECT_DNA_Study_Library-(ok-chart)

import csv
import glob
from gedcom.element.individual import IndividualElement
from gedcom.parser import Parser
import pandas as pd
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side, Alignment, PatternFill
import os

# Hard-coded interval scheme
interval_scheme = [
    (1000, 1), (1025, 2), (1050, 3), (1075, 4), (1100, 5),
    (1125, 6), (1150, 7), (1175, 8), (1200, 9), (1225, 10),
    (1250, 11), (1275, 12), (1300, 13), (1325, 14), (1350, 15),
    (1375, 16), (1400, 17), (1425, 18), (1450, 19), (1475, 20),
    (1500, 21), (1525, 22), (1550, 23), (1575, 24), (1600, 25),
    (1625, 26), (1650, 27), (1675, 28), (1700, 29), (1725, 30),
    (1750, 31), (1775, 32), (1800, 33), (1825, 34), (1850, 35),
    (1875, 36), (1900, 37), (1925, 38), (1950, 39), (1975, 40),
    (2000, 41), (2025, 42), (2050, 43), (2075, 44)
]

# Function to assign interval based on birthdate
def assign_interval(birth_date, interval_scheme):
    if pd.isnull(birth_date):
        return 99
    try:
        birth_year = int(birth_date[-4:])
    except ValueError:
        return 99
    for year, interval in interval_scheme:
        if birth_year <= year:
            return interval
    return 99

class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}
        self.anchor_gen1 = None  # Initialize anchor_gen1 here

    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value

    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '')
        parts = name.split('/', 1)
        first_name = parts[0].split(' ')[0]
        last_name = parts[1].rstrip('/') if len(parts) > 1 else ""
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        global anchor_gen1  # Declare that we're using the global variable
        anchor_gen1 = self.anchor_gen1  # Update the global variable
        return self.gen_person.strip('@')

    def get_anchor_gen1(self):
        return self.anchor_gen1

    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')

    def get_extractable_cm(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            cm_value = npfx_value.split('&')[0].strip()
        else:
            cm_value = npfx_value.strip()
        try:
            int(cm_value)
            return cm_value
        except ValueError:
            return 'error'

    def get_extractable_sort(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            sort_value = npfx_value.split('&')[1].strip()
            return sort_value
        else:
            return ''

    def get_extractable_FAMC(self):
        return self.extractable_detail.get('FAMC', '').strip('@')

    def get_birth_date(self):
        return self.extractable_detail.get('BIRTH_DATE', '')

    def get_fams(self):
        return self.extractable_detail.get('FAMS', '').strip('@')

# Set of excluded record IDs
excluded_ids = {
    'I47640', 'I55585', 'I47666', 'I55586', 'I47570', 'I55587',
    'I47569', 'I47641', 'I47571', 'I47572', 'I47549', 'I47550',
    'I47573', 'I55588', 'I47548', 'I47551', 'I47553', 'I24502', 'I24503',
    'I47659', 'I48070', 'I47660', 'I48129', 'I48130', 'I48126', 'I48127'
}

# Function definitions
def extract_id(record):
    id_start = record.find('@') + 1
    id_end = record.find('@', id_start)
    return record[id_start:id_end]

def extract_name(record):
    name_start = record.find('1 NAME ') + 6
    name_end = record.find('\n', name_start)
    name = record[name_start:name_end]
    first_name, last_name = name.split('/', 1)
    first_name = first_name[:30]
    last_name = last_name[:30].rstrip('/')
    return last_name.replace(" ", "") + first_name.replace(" ", "")

name_to_id = {}   # Global dictionary to hold name to ID mapping

class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []

    @staticmethod
    def get_standard_name(file_path):
        file_name = file_path.split('/')[-1]
        if '.' in file_name:
            file_name = file_name.rsplit('.', 1)[0]
        standard_name = file_name.replace(' ', '_').lower()
        return standard_name

    def parse_gedcom(self):
        global name_to_id  # Declare name_to_id as global to modify it
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            gedcom_lines = f.readlines()
        current_dataset = None
        npfx_count = 0
        total_count = 0

        for line in gedcom_lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0])
            tag = parts[1]
            value = parts[2] if len(parts) > 2 else None

            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                individual_id = tag.strip('@')
                if individual_id in excluded_ids:
                    continue  # Skip processing for the excluded IDs

                total_count += 1
                current_dataset = GedcomDataset(tag)
                self.gedcom_datasets.append(current_dataset)

                # Populate name_to_id
                individual_name = current_dataset.get_anchor_gen1()
                individual_id = current_dataset.get_gen_person()
                name_to_id[individual_name] = individual_id

            elif current_dataset is not None:
                if level == 1 and tag in ['NAME', 'FAMC', 'FAMS', 'BIRT', 'SEX']:
                    current_key = tag
                    current_dataset.add_extractable_detail(current_key, value)

                elif level == 2 and tag == 'DATE' and current_key == 'BIRT':
                    current_dataset.add_extractable_detail('BIRTH_DATE', value)

                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1
                    current_dataset.add_extractable_detail(tag, value)

        print(f'GEDCOM contained {total_count} total records')
        print(f'Records tagged and filtered by NPFX: {npfx_count}')

        # First level of filtering: Filter those with NPFX
        for dataset in self.gedcom_datasets:
            if dataset.get_extractable_NPFX():
                self.filter_pool.append(dataset)

        # Remove excluded IDs from filter_pool
        self.filter_pool = [ds for ds in self.filter_pool if ds.get_gen_person() not in excluded_ids]

        # Check if manual filtering should be applied
        manual_filter_activated = True  # or False depending on your situation

        # Second level of filtering: Apply manual filter from Excel sheet
        if manual_filter_activated:
            try:
                df = pd.read_excel('filtered_ids.xlsx')
            except FileNotFoundError:
                print("filtered_ids.xlsx not found. Skipping second-level manual filter.")
            else:
                manual_filtered_ids = set(df['ID'])
                print(f"Manual filter IDs loaded: {len(manual_filtered_ids) - 1}")

                self.filter_pool = [dataset for dataset in self.filter_pool if dataset.get_gen_person() in manual_filtered_ids]
                print(f"After manual filter, total records: {len(self.filter_pool)}")

    def apply_manual_filter(self):
        manual_filter_activated = True
        if manual_filter_activated:
            try:
                df = pd.read_excel('filtered_ids.xlsx')
                manual_filtered_ids = set(df['ID'].astype(str))  # Ensure IDs are strings
                print(f"Manual filter IDs loaded: {len(manual_filtered_ids)}")
            except FileNotFoundError:
                print("filtered_ids.xlsx not found. Skipping second-level manual filter.")
            else:
                # Debug output to verify IDs before filtering
                print("IDs before manual filter:", [ds.get_gen_person() for ds in self.filter_pool][:10])
                self.filter_pool = [ds for ds in self.filter_pool if str(ds.get_gen_person()) in manual_filtered_ids]
                print("IDs after manual filter:", [ds.get_gen_person() for ds in self.filter_pool][:10])
                print(f"After manual filter, total records: {len(self.filter_pool)}")

    def check_and_apply_exclusion_filter(self):
        """Apply exclusion filter if '/exclude_ids.xlsx' is present."""
        file_path = '/content/exclude_ids.xlsx'  # Updated path
        if os.path.exists(file_path):
            try:
                df_exclude = pd.read_excel(file_path)
                if 'ID' in df_exclude.columns:
                    exclude_ids = set(df_exclude['ID'].astype(str))  # Ensure conversion to string
                    print(f"Exclusion filter IDs loaded: {len(exclude_ids)}")
                    print(f"Sample of IDs to exclude: {list(exclude_ids)[:5]}")  # Print some sample IDs
                    # Apply the exclusion filter
                    initial_count = len(self.filter_pool)
                    self.filter_pool = [ds for ds in self.filter_pool if str(ds.get_gen_person()) not in exclude_ids]
                    print(f"Excluded {initial_count - len(self.filter_pool)} records based on exclusion IDs.")
                else:
                    print("Column 'ID' not found in the Excel file.")
            except Exception as e:
                print(f"Failed to apply exclusion filter: {str(e)}")
        else:
            print(f"No exclusion filter applied, '{file_path}' not found. Check the path and ensure the file is uploaded to Colab.")

def input_prime_surname(last_prime_surname=None):
    if last_prime_surname:
        last_name = input(f"Enter prime_surname (default: {last_prime_surname}): ")
        if not last_name:
            last_name = last_prime_surname
    else:
        last_name = input("Enter prime_surname: ")
    return last_name

def select_gedcom_file():
    gedcom_files = glob.glob('*.ged')
    if not gedcom_files:
        print("No GEDCOM files found.")
        return None

    print("Automatically selecting the first GEDCOM file.")
    return gedcom_files[0]

    while True:
        try:
            selected_num = int(input("Enter the number of the GEDCOM file you want to use: "))
            if 1 <= selected_num <= len(gedcom_files):
                return gedcom_files[selected_num - 1]
            else:
                print("Invalid number. Please enter a valid number from the list.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

gedcom_file_path = select_gedcom_file() # Call the function to let the user select the GEDCOM file
if gedcom_file_path:
    # Use the selected GEDCOM file path to create an instance of the Gedcom class
    gedcom_instance = Gedcom(gedcom_file_path)
    gedcom_instance.parse_gedcom()

    individuals = []  # Initialize the list of individuals

    for dataset in gedcom_instance.filter_pool:    # Iterate over the filter_pool list, add each last name and ID to list
        individual_id = dataset.get_gen_person()
        last_name = dataset.get_anchor_gen1()
        individuals.append((last_name, individual_id))

    print(f'Records tagged and filtered by NPFX: {len(individuals)}')

    with open(gedcom_file_path, 'r') as file:    # Read the GEDCOM file and split it into individual and family records
        data = file.read()
    data = data.split('\n0 ')
    records = {extract_id(record): record for record in data}

def has_both_parents(records, mother_id, father_id):
    return mother_id in records and father_id in records

visited_pairs = set()
generation_table = []

# Function to find parents, ensuring excluded IDs are not processed
def find_parents(individual_id, generation, records):
    if individual_id in excluded_ids or individual_id not in records:
        return
    record = records[individual_id]
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]
    if famc_id not in records:
        return

    fam_record = records[famc_id]
    wife_start = fam_record.find('1 WIFE @') + 8
    wife_end = fam_record.find('@', wife_start)
    mother_id = fam_record[wife_start:wife_end]

    husb_start = fam_record.find('1 HUSB @') + 8
    husb_end = fam_record.find('@', husb_start)
    father_id = fam_record[husb_start:husb_end]

    if mother_id and mother_id in records and father_id and father_id in records:
        parent_pair = (father_id, mother_id)
        if parent_pair not in visited_pairs:
            visited_pairs.add(parent_pair)
            generation_table.append((generation, parent_pair))

    if mother_id:
        find_parents(mother_id, generation + 1, records)

    if father_id:
        find_parents(father_id, generation + 1, records)

# Function to find distant ancestors, ensuring excluded IDs are not processed
def find_distant_ancestors(individual_id, records, path=None):
    if individual_id in excluded_ids:
        return []
    path = path if path is not None else []
    if path is None:
        path = [individual_id]
    else:
        path.append(individual_id)

    if individual_id not in records:
        return []

    record = records[individual_id]
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]

    if famc_id not in records:
        return [path]

    fam_record = records[famc_id]
    wife_start = fam_record.find('1 WIFE @') + 8
    wife_end = fam_record.find('@', wife_start)
    mother_id = fam_record[wife_start:wife_end]

    husb_start = fam_record.find('1 HUSB @') + 8
    husb_end = fam_record.find('@', husb_start)
    father_id = fam_record[husb_start:husb_end]

    if father_id is None and mother_id is None:
        return [path]

    paths = []
    if father_id and father_id not in excluded_ids:
        new_path = list(path)
        paths.extend(find_distant_ancestors(father_id, records, new_path))

    if mother_id and mother_id not in excluded_ids:
        new_path = list(path)
        paths.extend(find_distant_ancestors(mother_id, records, new_path))

    return paths

# Other parts of the script remain the same

# Example usage after parsing and filtering
gedcom_instance.parse_gedcom()

# Save the filter_pool to an Excel file
#print_filter_pool_to_excel(gedcom_instance.filter_pool)

filtered_datasets = gedcom_instance.filter_pool

def calculate_score(distant_ancestors_paths, records):
    name_paths = []
    for path in distant_ancestors_paths:
        name_path = [extract_name(records.get(id, '')) for id in path]
        name_paths.append(name_path)

    path_scores = {}
    for idx, name_path in enumerate(name_paths):
        score = 0
        for generation, name in enumerate(name_path):
            if 'Yates' in name:
                score += 1 * (generation + 1)
        path_scores[idx] = score

    if path_scores:
        winning_path_index = max(path_scores, key=path_scores.get)
        winning_path_score = path_scores[winning_path_index]
        winning_path_names = name_paths[winning_path_index]
        winning_path_ids = distant_ancestors_paths[winning_path_index]
    else:
        winning_path_index = None
        winning_path_score = 0
        winning_path_names = []
        winning_path_ids = []

    return winning_path_score, winning_path_names, winning_path_ids

def filter_ancestral_line(winning_path_ids, generation_table):
    matching_table = []
    for generation, pair in generation_table:
        id1, id2 = pair
        if id1 in winning_path_ids or id2 in winning_path_ids:
            matching_table.append((generation, pair))
    return matching_table

def process_individual(individual_id, gedcom_instance, records, interval_scheme):
    global generation_table
    global visited_pairs
    global anchor_gen1  # Declare that we're using the global variable

    generation_table = []
    visited_pairs = set()

    find_parents(individual_id, 1, records)
    distant_ancestors_paths = find_distant_ancestors(individual_id, records)
    winning_path_score, winning_path_names, winning_path_ids = calculate_score(distant_ancestors_paths, records)
    filtered_ancestral_line = filter_ancestral_line(winning_path_ids, generation_table)
    filtered_ancestral_line.sort(key=lambda x: x[0])
    filtered_ancestral_line_names = []
    birth_date = None  # Initialize birth_date

    # Extract cm_value, sort_value, anchor_gen1, and birth_date
    for dataset in gedcom_instance.filter_pool:
        if dataset.get_gen_person() == individual_id:
            cm_value = dataset.get_extractable_cm()
            sort_value = dataset.get_extractable_sort()
            anchor_gen1 = dataset.get_anchor_gen1()  # Update anchor_gen1 locally here
            birth_date = dataset.extractable_detail.get('BIRTH_DATE')  # Get birth date
            break
    else:
        cm_value = 'N/A'
        sort_value = 'N/A'
        birth_date = 'N/A'  # Set default if not found

    if anchor_gen1 is not None:
        filtered_ancestral_line_names.insert(0, anchor_gen1)

    ancestors_data = []

    # Extract ancestor information and concatenate name with Date Interval
    for generation, pair in filtered_ancestral_line:
        ancestor_details = []
        for ancestor_id in pair:
            if ancestor_id in records:
                ancestor_record = records[ancestor_id]
                ancestor_name = extract_name(ancestor_record)
                ancestor_sex = 'M' if '1 SEX M' in ancestor_record else 'F' if '1 SEX F' in ancestor_record else ''
                birth_date_start = ancestor_record.find('2 DATE ') + 7
                birth_date_end = ancestor_record.find('\n', birth_date_start)
                ancestor_birth_date = ancestor_record[birth_date_start:birth_date_end].strip() if birth_date_start != -1 else 'N/A'
                ancestor_date_interval = assign_interval(ancestor_birth_date, interval_scheme)
                ancestor_details.append({
                    'ID': ancestor_id,
                    'Name': ancestor_name,
                    'Sex': ancestor_sex,
                    'Birth Date': ancestor_birth_date,
                    'Date Interval': ancestor_date_interval
                })
        if ancestor_details:
            name_with_interval_1 = f"{ancestor_details[0]['Date Interval']}{ancestor_details[0]['Name']}"
            name_with_interval_2 = f"{ancestor_details[1]['Date Interval']}{ancestor_details[1]['Name']}" if len(ancestor_details) > 1 else '99Unknown'
            combined_name_with_interval = f"{name_with_interval_1}&{name_with_interval_2}"
            ancestors_data.append(combined_name_with_interval)

    # Reverse the order to start with the oldest ancestor
    ancestors_data.reverse()
    filtered_ancestral_line_str = "~~~".join(ancestors_data)

    # Assign date interval
    date_interval = assign_interval(birth_date, interval_scheme)

    individual_data = {
        'ID': individual_id,
        'Name': extract_name(records[individual_id]),
        'Sex': dataset.extractable_detail.get('SEX', ''),
        'Birth Date': birth_date,
        'Date Interval': date_interval,
        'FAMS': dataset.get_fams(),
        'FAMC': dataset.get_extractable_FAMC(),
        'cM': cm_value,
        'Sort': sort_value,
        'Ancestors': filtered_ancestral_line_str  # Add ancestors data to individual data
    }

    return individual_data, filtered_ancestral_line_str

# Initialize the dictionary to store individual data
DNA_Study_Library = {}

# Process individuals and populate the dictionary
for dataset in gedcom_instance.filter_pool:  # Assuming filter_pool is iterable
    individual_id = dataset.get_gen_person()

    # Reset global variables for each new individual
    visited_pairs.clear()
    generation_table = []

    # Process Individual and Get Data
    individual_data, filtered_ancestral_line_str = process_individual(individual_id, gedcom_instance, records, interval_scheme)

    # Store the data in the dictionary
    DNA_Study_Library[individual_id] = individual_data

# Output the first 5 records in a columnar format for validation
print(f"{'ID':<6}{'Name':<20}{'Sort':<20}{'cM':<5}{'Ancestors':<30}")
for idx, (individual_id, data) in enumerate(DNA_Study_Library.items()):
    if idx >= 1:
        break
    # Replace None values with empty strings
    name = data['Name'] or ''
    sex = data['Sex'] or ''
    date_interval = data['Date Interval'] or ''
    fams = data['FAMS'] or ''
    famc = data['FAMC'] or ''
    cm = data['cM'] or ''
    sort_value = data['Sort'] or ''
    ancestors = data['Ancestors']
    print(f"{individual_id:<7}{name:<25}{sort_value:<15}{cm:<5}{ancestors:<30}")

# Function to save the data to an Excel file
def save_to_excel(data_dict, filename):
    # Convert the dictionary to a pandas DataFrame
    df = pd.DataFrame.from_dict(data_dict, orient='index')

    # Save the DataFrame to an Excel file using openpyxl
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='DNA_Study_Library', index=False)  # index=False to prevent writing the index

        # Apply some basic formatting
        workbook = writer.book
        worksheet = writer.sheets['DNA_Study_Library']

        # Adjust column width
        for column in worksheet.columns:
            max_length = 0
            column = [cell for cell in column]
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2)
            worksheet.column_dimensions[column[0].column_letter].width = adjusted_width

# Save the DNA Study Library to an Excel file
save_to_excel(DNA_Study_Library, 'DNA_Study_Library.xlsx')

print("Data has been saved to DNA_Study_Library.xlsx")


Automatically selecting the first GEDCOM file.
GEDCOM contained 55704 total records
Records tagged and filtered by NPFX: 1045
filtered_ids.xlsx not found. Skipping second-level manual filter.
Records tagged and filtered by NPFX: 1045
GEDCOM contained 55704 total records
Records tagged and filtered by NPFX: 1045
filtered_ids.xlsx not found. Skipping second-level manual filter.
ID    Name                Sort                cM   Ancestors                     
I73    YatesJamesRobert         yates,ronald   1047 30YatesBenjamin&42SearchingStill~~~31YatesBenjamin&42SearchingStill~~~32YatesJohn&42SearchingStill~~~33YatesJohn&33SwiftMary~~~34YatesJohnE&34RobersonElizabeth~~~36YatesJamesWilson&36OttMaryEllen~~~37YatesHarryElmer&38TingleyOrliaGeorgeann
Data has been saved to DNA_Study_Library.xlsx


In [17]:
# /content/EXP#4_chart_black_separators.xlsx

import pandas as pd
from openpyxl import Workbook, drawing
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side, Alignment, PatternFill

def parse_ancestors(df):
    """Parse the Ancestors column to determine the first parent pair."""
    first_parents = []
    for index, row in df.iterrows():
        nodes = row['Ancestors'].split('~~~')
        if nodes:
            first_parent_pair = nodes[0]
            first_parents.append(first_parent_pair)
    return set(first_parents)

def process_data(filepath):
    """Load data, process it to expand 'Ancestors', calculate FQ and QI, and return a dictionary of results."""
    # Load the DataFrame
    df = pd.read_excel(filepath)

    # Fill NaN values in 'Ancestors' column with empty strings and ensure all values are strings
    df['Ancestors'] = df['Ancestors'].fillna('').astype(str)

    # Parse the Ancestors column to get the first parent pairs
    first_parents = parse_ancestors(df)

    # Prepare to expand and process
    expanded_data = []

    # Process each row to expand and mark root nodes
    for index, row in df.iterrows():
        nodes = row['Ancestors'].split('~~~')
        for i, node in enumerate(nodes):
            parent = nodes[i-1] if i > 0 else None
            expanded_data.append({
                'Parents': parent,
                'Offspring & Spouse': node,
                'cM': row['cM'],
                'ID': row.get('ID', '')  # Ensure ID is optionally included
            })

    # Convert the list to a DataFrame
    expanded_df = pd.DataFrame(expanded_data)

    # Group by 'Parents' and 'Offspring & Spouse' and calculate FQ and QI
    grouped_df = expanded_df.groupby(['Parents', 'Offspring & Spouse']).agg(
        FQ=('Offspring & Spouse', 'size'),
        cM_sum=('cM', 'sum')
    ).reset_index()

    # Calculate QI as the sum of all cM values divided by FQ, rounded to an integer
    grouped_df['QI'] = (grouped_df['cM_sum'] / grouped_df['FQ']).round().astype(int)

    # Mark root nodes based on first parent pairs
    grouped_df['Root'] = grouped_df.apply(lambda x: 'Yes' if x['Parents'] in first_parents else '', axis=1)

    # Select relevant columns and filter results where FQ is greater than or equal to 8
    final_results_df = grouped_df[['Root', 'Parents', 'FQ', 'QI', 'Offspring & Spouse']]
    final_results_df = final_results_df[final_results_df['FQ'] >= 8].copy()

    # Extract the prefix for segmenting the parents
    final_results_df['Prefix'] = final_results_df['Parents'].str.extract(r'(\d+)')

    # Ensure the Prefix column is treated as a string
    final_results_df['Prefix'] = final_results_df['Prefix'].astype(str)

    return final_results_df

def sort_within_segment(group):
    """Sort the group by the FQ in descending order within each segment."""
    return group.sort_values(by='FQ', ascending=False)

def sort_all_segments(data_dict):
    """Sort all segments within the data based on the FQ in descending order."""
    df = pd.DataFrame(data_dict)

    # Apply sorting within each segment
    sorted_df = df.groupby('Prefix', group_keys=False).apply(sort_within_segment).reset_index(drop=True)

    # Drop the Prefix column as it is no longer needed
    sorted_df = sorted_df.drop(columns=['Prefix'])

    return sorted_df

def visualize_data(sorted_df):
    """Visualize the sorted data in an Excel file with styled segments."""
    # Create an Excel file with openpyxl
    wb = Workbook()
    ws = wb.active

    # Adding rows to worksheet
    headers = ['Root', 'Parents', 'FQ', 'QI', 'Offspring & Spouse']
    ws.append(headers)

    # Center-align the header labels
    header_alignment = Alignment(horizontal="center", vertical="center")
    for cell in ws[1]:
        cell.alignment = header_alignment

    # Initialize variables to track last seen parent initial and generation
    last_parent_initial = None
    parent_seen = set()

    # Adding the rows and assigning interval offset for staggered node-parents
    interval_offset = 0
    for r in dataframe_to_rows(sorted_df, index=False, header=False):
        current_parent = r[1]

        # Calculate interval offset
        if current_parent not in parent_seen:
            parent_seen.add(current_parent)
            current_parent_prefix = current_parent[:2] if current_parent else ''
            if last_parent_initial is not None and current_parent_prefix != last_parent_initial:
                # Insert a black row at the end of the previous segment
                ws.append([''] * len(headers))
                black_row_index = ws.max_row
                ws.row_dimensions[black_row_index].height = 5
                for cell in ws[black_row_index]:
                    cell.fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')
            last_parent_initial = current_parent_prefix
            ws.append([*r])
        else:
            r[1] = ''  # Clear parent name for repeated entries within the same generation
            ws.append([r[0], '', *r[2:]])

    # Add a black row at the end of the last segment
    ws.append([''] * len(headers))
    black_row_index = ws.max_row
    ws.row_dimensions[black_row_index].height = 5
    for cell in ws[black_row_index]:
        cell.fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')

    # Adding borders and alignment for readability
    thin = Side(border_style="thin", color="000000")
    center_alignment = Alignment(horizontal="center", vertical="center")
    right_alignment = Alignment(horizontal="right", vertical="center")
    left_alignment = Alignment(horizontal="left", vertical="center")

    for row in ws.iter_rows(min_row=2, max_col=len(headers), max_row=ws.max_row):
        for cell in row:
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        row[1].alignment = right_alignment  # Right-align Parents column
        row[4].alignment = left_alignment  # Left-align Offspring & Spouse column

    # Replace 'Yes' with an arrow image
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=1):
        for cell in row:
            if cell.value == 'Yes':
                if ws.cell(row=cell.row, column=2).value:  # Check if the Parents column is not blank
                    img = drawing.image.Image('arrow.png')
                    img.width, img.height = (25, 25)  # Increase size to 25x25
                    img.anchor = cell.coordinate
                    ws.add_image(img)
                cell.value = ''  # Clear the cell value

    # Save the workbook
    output_file = '/content/EXP#4_chart_black_separators.xlsx'
    wb.save(output_file)
    print(f"Results successfully saved with styled colors to {output_file}")

# Specify the file path for the data
input_file_path = '/content/DNA_Study_Library.xlsx'  # Replace with the correct path

# Process the data and save it in a dictionary
processed_data_dict = process_data(input_file_path).to_dict(orient='list')

# Sort all segments within the data
sorted_data = sort_all_segments(processed_data_dict)

# Visualize the sorted data
visualize_data(sorted_data)



#*********************************************************************************GOOD

Results successfully saved with styled colors to /content/EXP#4_chart_black_separators.xlsx


In [16]:
#*****************************************  Good Trailing Descendants

import pandas as pd
from openpyxl import Workbook, drawing
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side, Alignment, PatternFill

def parse_ancestors(df):
    """Parse the Ancestors column to determine the first parent pair."""
    first_parents = []
    for index, row in df.iterrows():
        nodes = row['Ancestors'].split('~~~')
        if nodes:
            first_parent_pair = nodes[0]
            first_parents.append(first_parent_pair)
    return set(first_parents)

def process_data(filepath):
    """Load data, process it to expand 'Ancestors', calculate FQ and QI, and return a dictionary of results."""
    # Load the DataFrame
    df = pd.read_excel(filepath)

    # Fill NaN values in 'Ancestors' column with empty strings and ensure all values are strings
    df['Ancestors'] = df['Ancestors'].fillna('').astype(str)

    # Parse the Ancestors column to get the first parent pairs
    first_parents = parse_ancestors(df)

    # Prepare to expand and process
    expanded_data = []

    # Process each row to expand and mark root nodes
    for index, row in df.iterrows():
        nodes = row['Ancestors'].split('~~~')
        for i, node in enumerate(nodes):
            parent = nodes[i-1] if i > 0 else None
            expanded_data.append({
                'Parents': parent,
                'Offspring & Spouse': node,
                'cM': row['cM'],
                'ID': row.get('ID', '')  # Ensure ID is optionally included
            })

    # Convert the list to a DataFrame
    expanded_df = pd.DataFrame(expanded_data)

    # Group by 'Parents' and 'Offspring & Spouse' and calculate FQ and QI
    grouped_df = expanded_df.groupby(['Parents', 'Offspring & Spouse']).agg(
        FQ=('Offspring & Spouse', 'size'),
        cM_sum=('cM', 'sum')
    ).reset_index()

    # Calculate QI as the sum of all cM values divided by FQ, rounded to an integer
    grouped_df['QI'] = (grouped_df['cM_sum'] / grouped_df['FQ']).round().astype(int)

    # Mark root nodes based on first parent pairs
    grouped_df['Root'] = grouped_df.apply(lambda x: 'Yes' if x['Parents'] in first_parents else '', axis=1)

    # Select relevant columns and filter results where FQ is greater than or equal to 8
    final_results_df = grouped_df[['Root', 'Parents', 'FQ', 'QI', 'Offspring & Spouse']]
    final_results_df = final_results_df[final_results_df['FQ'] >= 8].copy()

    # Extract the prefix for segmenting the parents
    final_results_df['Prefix'] = final_results_df['Parents'].str.extract(r'(\d+)')

    # Ensure the Prefix column is treated as a string
    final_results_df['Prefix'] = final_results_df['Prefix'].astype(str)

    return final_results_df, len(df)

def sort_within_segment(group):
    """Sort the group by the FQ in descending order within each segment."""
    return group.sort_values(by='FQ', ascending=False)

def sort_all_segments(data_dict):
    """Sort all segments within the data based on the FQ in descending order."""
    df = pd.DataFrame(data_dict)

    # Apply sorting within each segment
    sorted_df = df.groupby('Prefix', group_keys=False).apply(sort_within_segment).reset_index(drop=True)

    # Drop the Prefix column as it is no longer needed
    sorted_df = sorted_df.drop(columns=['Prefix'])

    return sorted_df

def assemble_descendants(df, total_records):
    """Assemble descendants starting from root parents and include FQ, QI, and % Total."""
    root_parents = df[df['Root'] == 'Yes']['Parents'].unique()
    assembled_data = []

    for root in root_parents:
        current_generation = [(root, 0)]
        assembled_data.append({
            'Root': 'Yes',
            'Parents and Trailing Descendants': root,
            'FQ': df[df['Parents'] == root]['FQ'].values[0],
            'QI': df[df['Parents'] == root]['QI'].values[0],
            '% Total': f"{round(df[df['Parents'] == root]['FQ'].values[0] / total_records * 100)}%"
        })
        while current_generation:
            next_generation = []
            for parent, depth in current_generation:
                children = df[df['Parents'] == parent]['Offspring & Spouse'].tolist()
                for i, child in enumerate(children):
                    fq = df[(df['Parents'] == parent) & (df['Offspring & Spouse'] == child)]['FQ'].values[0]
                    qi = df[(df['Parents'] == parent) & (df['Offspring & Spouse'] == child)]['QI'].values[0]
                    percent_total = f"{round(fq / total_records * 100)}%"
                    descendant = f"{'  ' * (depth + 1)}→ {child}"
                    assembled_data.append({
                        'Root': '',
                        'Parents and Trailing Descendants': descendant,
                        'FQ': fq,
                        'QI': qi,
                        '% Total': percent_total
                    })
                    next_generation.append((child, depth + 1))
            current_generation = next_generation

    return pd.DataFrame(assembled_data)

def save_to_excel_with_styles(df, output_path):
    """Save the assembled data to an Excel file with the desired format and styles."""
    # Create an Excel file with openpyxl
    wb = Workbook()
    ws = wb.active

    # Adding rows to worksheet
    headers = ['Root', 'Parents and Trailing Descendants', 'FQ', 'QI', '% Total']
    ws.append(headers)

    # Center-align the header labels
    header_alignment = Alignment(horizontal="center", vertical="center")
    for cell in ws[1]:
        cell.alignment = header_alignment

    # Initialize variables to track last seen parent initial and generation
    parent_seen = set()
    previous_root = None

    for r in dataframe_to_rows(df, index=False, header=False):
        current_parent = r[1].strip().split('→')[0]

        # Insert a black row at the end of each root parent's segment
        if r[0] == 'Yes' and previous_root is not None:
            ws.append([''] * len(headers))
            black_row_index = ws.max_row
            ws.row_dimensions[black_row_index].height = 5
            for cell in ws[black_row_index]:
                cell.fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')

        # Add the row to the worksheet
        ws.append(r)
        if r[0] == 'Yes':
            parent_seen.add(current_parent)
            previous_root = current_parent

    # Add a black row at the end of the last segment
    ws.append([''] * len(headers))
    black_row_index = ws.max_row
    ws.row_dimensions[black_row_index].height = 5
    for cell in ws[black_row_index]:
        cell.fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')

    # Adding borders and alignment for readability
    thin = Side(border_style="thin", color="000000")
    left_alignment = Alignment(horizontal="left", vertical="center")

    for row in ws.iter_rows(min_row=2, max_col=len(headers), max_row=ws.max_row):
        for cell in row:
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        row[1].alignment = left_alignment

    # Add red arrow images for root nodes
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=1):
        for cell in row:
            if cell.value == 'Yes':
                img = drawing.image.Image('/content/arrow.png')
                img.width, img.height = (25, 25)
                img.anchor = cell.coordinate
                ws.add_image(img)
                cell.value = ''  # Clear the cell value

    # Save the workbook
    wb.save(output_path)
    print(f"Results successfully saved to {output_path}")

# Specify the file path for the data
input_file_path = '/content/DNA_Study_Library.xlsx'  # Replace with the correct path

# Process the data and save it in a dictionary
processed_data, total_records = process_data(input_file_path)

# Sort all segments within the data
sorted_data = sort_all_segments(processed_data.to_dict(orient='list'))

# Assemble descendants into a DataFrame
descendants_df = assemble_descendants(pd.DataFrame(sorted_data), total_records)

# Save the assembled data to an Excel file with styles
output_file_path = '/content/trailing_descendants.xlsx'
save_to_excel_with_styles(descendants_df, output_file_path)

# Print records tagged and filtered by NPFX
print(f"Records tagged and filtered by NPFX: {total_records}")


#*****************************************  Good Trailing Descendants

Results successfully saved to /content/trailing_descendants.xlsx
